In [1]:
import ciropt as co

In [2]:
L_smooth = 1.
mu = 0.001

Inductance = 5.
R = 1.

n_func = 3
solver = "ca"
# solver = "ca_canonical"
# solver = "ca_canonical_X"

# Ciropt problem

In [3]:
num_fi = 3
problem = co.admm_consensus(num_fi, mu, L_smooth, R, Inductance)

res, opti = problem.solve(solver = solver, verbose=True)[:2]
ca_vars = problem.vars

dim_G=14, dim_F=12
opts={'ipopt.max_iter': 50000}

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:    12458
Number of nonzeros in inequality constraint Jacobian.:      122
Number of nonzeros in Lagrangian Hessian.............:      652

Total number of variables............................:      340
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints......

In [4]:
res

{'b': -2.589691941730328e-10,
 'h': 6.26531158956202e-10,
 'alpha': 0.00021402841846522824,
 'beta': 0.2026833387193412}

# PEP verification

In [5]:
import PEPit
from PEPit.functions import SmoothStronglyConvexFunction, SmoothConvexFunction, ConvexFunction
from PEPit.primitive_steps import proximal_step

b = res["b"] 
if "d" in res: d = res["d"]
else: d = 0
h = res["h"]
alpha = res["alpha"]
beta = res["beta"]

In [6]:
problem = PEPit.PEP()

fs = [0] * n_func
for i in range(n_func):
    fs[i] = problem.declare_function(
                    SmoothStronglyConvexFunction,
                    mu=mu,   
                    L=L_smooth) 
    if i == 0: f = fs[i]
    else: f += fs[i]


x_star = f.stationary_point()
g_star, f_star = f.oracle(x_star)

gs_star = [0] * n_func
for i in range(n_func):
    gi, fi = fs[i].oracle(x_star)
    gs_star[i] = gi

y_Ls_1 = [0] * n_func
for i in range(n_func-1):
    y_Ls_1[i] = problem.set_initial_point()
    if i == 0: y_Ls_1[-1] = -y_Ls_1[i]
    else: y_Ls_1[-1] -= y_Ls_1[i]
z_0 = problem.set_initial_point()

triplets_1 = [0] * n_func
for i in range(n_func):
    xi, gi, fi = proximal_step((R * y_Ls_1[i] + z_0), fs[i], R)
    if i == 0: z_1 = xi
    else: z_1 += xi
    triplets_1[i] = (xi, gi, fi)
z_1 = z_1 / n_func


y_Ls_1p5 = [0] * n_func
for i in range(n_func):
    y_Ls_1p5[i] = y_Ls_1[i] + (alpha * h / Inductance) * (z_1 - triplets_1[i][0]) 
triplets_1p5 = [0] * n_func
for i in range(n_func):
    xi, gi, fi = proximal_step((R * y_Ls_1p5[i] + z_1), fs[i], R)
    if i == 0: z_1p5 = xi
    else: z_1p5 += xi
    triplets_1p5[i] = (xi, gi, fi)

z_1p5 = z_1p5 / n_func
y_Ls_2 = [0] * n_func
for i in range(n_func):
    y_Ls_2[i] = y_Ls_1[i] + (beta * h / Inductance) * (z_1 - triplets_1[i][0]) \
              + ((1 - beta) * h / Inductance) * (z_1p5 - triplets_1p5[i][0])
triplets_2 = [0] * n_func
for i in range(n_func):
    xi, gi, fi = proximal_step((R * y_Ls_1p5[i] + z_1p5), fs[i], R)
    if i == 0: z_2 = xi
    else: z_2 += xi
    triplets_2[i] = (xi, gi, fi)
z_2 = z_2 / n_func

# Set the performance metric to the function values accuracy
E_1 = 0; E_2 = 0; f_1 = 0
E_1 = 0; E_2 = 0; f_1 = 0
for i in range(n_func):
    E_1 += (Inductance/2) * (y_Ls_1[i] - gs_star[i]) ** 2
    E_2 += (Inductance/2) * (y_Ls_2[i] - gs_star[i]) ** 2
    f_1 += triplets_1[i][2]
    if i == 0: Delta_1 = d * R * (triplets_1[i][1] - y_Ls_1[i])**2 
    else: Delta_1 += d * R * (triplets_1[i][1] - y_Ls_1[i])**2 
Delta_1 += b * (f_1 - f_star)
problem.set_performance_metric(E_2 - (E_1 - Delta_1))

# Solve the PEP with verbose = 1
verbose = 0
diff = problem.solve(verbose=verbose)

print(f"{diff=}")

diff=-5.528771653897605e-09
